In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('../data/slurm_wrapper_ce5.log') as f:
    data5 = ([line.split() for line in f])

In [3]:
ce5 = pd.DataFrame([[x[0] + ' ' + x[1], x[4], float(x[10]), int(x[13]), x[16]] 
                       for x in data5], columns=['BEGIN', 'USER', 'DURATION', 'RC', 'COMMAND'])

In [4]:
ce5['value_grp'] = (ce5.RC.diff(1) != 0).cumsum()
# ce5['rc_grp'] = (ce5.RC.diff(1) != 0).cumsum()
# ce5['comm'] = ce5['COMMAND'].astype('category').cat.codes
# ce5['command_grp'] = (ce5.comm.diff(1) != 0).cumsum()

In [5]:
ce5 = ce5.reset_index()

In [6]:
f_index = pd.DataFrame({'Consecutive' : ce5.groupby('value_grp').size(),
                    'index' : ce5.groupby('value_grp').index.first(),
                    'dur' : ce5.groupby('value_grp').DURATION.first(),
                    'Returncode' : ce5.groupby('value_grp').RC.first()})

In [7]:
fe_index = f_index[(f_index['Returncode'] == 1) & (f_index['dur'] > 15)]

In [8]:
fe_df = ce5.join(fe_index[['index','Consecutive']].set_index('index'), how='inner').sort_values('BEGIN', ascending=False)

In [9]:
fe_df['COMMAND'].value_counts()

['/usr/bin/scontrol',    34694
['/usr/bin/sbatch',       7545
['/usr/bin/squeue',       3548
Name: COMMAND, dtype: int64

In [10]:
ce5[ce5['COMMAND'].str.contains('sacct')].index

Int64Index([      0,       1,       2,       3,       4,     125,     360,
                362,     428,     431,
            ...
            4770411, 4770449, 4770529, 4770562, 4770650, 4770690, 4770749,
            4770782, 4770855, 4770884],
           dtype='int64', length=357388)

In [11]:
sindex = np.array(ce5[ce5['COMMAND'].str.contains('sacct')].index)
sindex += -1
sc = ce5.join(pd.DataFrame(index = sindex), how='inner')
sc[sc['DURATION'] > 15]

,index,BEGIN,USER,DURATION,RC,COMMAND,value_grp
9118,9118,2020-10-16 16:00:16.038701,9204,18.315583,0,"['/usr/bin/scontrol',",245
10554,10554,2020-10-16 17:22:57.794663,9219,16.793026,0,"['/usr/bin/scontrol',",285
10599,10599,2020-10-16 17:24:13.928890,9202,15.571080,0,"['/usr/bin/scontrol',",285
11740,11740,2020-10-16 18:11:04.675942,9202,15.704717,0,"['/usr/bin/scontrol',",299
12440,12440,2020-10-16 18:47:50.962718,9202,17.149708,0,"['/usr/bin/scontrol',",309
...,...,...,...,...,...,...,...
4761560,4761560,2021-10-06 15:39:18.708410,9221,38.036971,1,"['/usr/bin/squeue',",271220
4761566,4761566,2021-10-06 15:39:19.632385,9221,38.039301,1,"['/usr/bin/squeue',",271222
4761568,4761568,2021-10-06 15:39:19.714047,9221,38.042240,1,"['/usr/bin/squeue',",271224
4761573,4761573,2021-10-06 15:39:19.882645,9221,38.072341,1,"['/usr/bin/squeue',",271226


In [12]:
ce5['COMMAND'].value_counts()

['/usr/bin/scontrol',    2813268
['/usr/bin/sbatch',       910534
['/usr/bin/scancel',      428813
['/usr/bin/sacct',        357388
['/usr/bin/squeue',       260888
['/usr/bin/squeue']            2
Name: COMMAND, dtype: int64

In [21]:
ce5[(ce5['RC'] == 1) & (ce5['DURATION'] > 15)]

,index,BEGIN,USER,DURATION,RC,COMMAND,value_grp
3465,3465,2020-10-16 11:43:45.753267,9201,20.150959,1,"['/usr/bin/sbatch',",104
3466,3466,2020-10-16 11:43:45.920205,9201,20.218138,1,"['/usr/bin/sbatch',",104
3467,3467,2020-10-16 11:43:45.939591,9201,20.188296,1,"['/usr/bin/sbatch',",104
3468,3468,2020-10-16 11:43:46.119672,9201,20.114305,1,"['/usr/bin/sbatch',",104
3469,3469,2020-10-16 11:43:46.380161,9202,20.092560,1,"['/usr/bin/sbatch',",104
...,...,...,...,...,...,...,...
4761573,4761573,2021-10-06 15:39:19.882645,9221,38.072341,1,"['/usr/bin/squeue',",271226
4761575,4761575,2021-10-06 15:39:19.936025,9221,38.052309,1,"['/usr/bin/squeue',",271228
4761576,4761576,2021-10-06 15:39:19.963830,9221,38.055701,1,"['/usr/bin/squeue',",271228
4761577,4761577,2021-10-06 15:39:20.070326,9221,38.051637,1,"['/usr/bin/squeue',",271228


In [23]:
f_index[(f_index['Returncode'] == 1) & (f_index['dur'] > 15)].describe()

,Consecutive,index,dur,Returncode
count,45787.000000,4.578700e+04,45787.000000,45787.0
mean,9.571385,2.334553e+06,21.577933,1.0
std,20.200997,1.215875e+06,6.109202,0.0
min,1.000000,3.465000e+03,15.021515,1.0
25%,1.000000,1.214044e+06,20.039559,1.0
50%,2.000000,2.165558e+06,20.047102,1.0
75%,6.000000,3.370722e+06,20.126347,1.0
max,900.000000,4.761585e+06,127.142892,1.0


In [22]:
f_index[(f_index['Returncode'] == 1)].describe()

,Consecutive,index,dur,Returncode
count,133349.000000,1.333490e+05,133349.000000,133349.0
mean,5.231880,2.368018e+06,7.680041,1.0
std,12.968102,1.300611e+06,10.725194,0.0
min,1.000000,3.570000e+02,0.016019,1.0
25%,2.000000,1.359994e+06,0.031451,1.0
50%,2.000000,2.201656e+06,0.280608,1.0
75%,3.000000,3.586745e+06,20.040108,1.0
max,900.000000,4.761585e+06,127.142892,1.0


In [19]:
f_index[(f_index['Returncode'] == 1) & (f_index['Consecutive'] == 1)]['Consecutive'].sum()

25522

In [29]:
fe_df[(fe_df['Consecutive'] == 900)]

,index,BEGIN,USER,DURATION,RC,COMMAND,value_grp,Consecutive
3004577,3004577,2021-05-18 06:32:07.287798,9221,19.054259,1,"['/usr/bin/scontrol',",183154,900


In [37]:
fe_list = list(fe_df.sort_values('Consecutive', ascending=False)['value_grp'])

In [42]:
for x in fe_list[:20]:
    print()
    print(ce5[ce5['value_grp'] == x].value_counts('COMMAND'))

COMMAND
['/usr/bin/sacct',       596
['/usr/bin/scontrol',    304
dtype: int64
COMMAND
['/usr/bin/sbatch',      310
['/usr/bin/scontrol',     74
dtype: int64
COMMAND
['/usr/bin/sbatch',      320
['/usr/bin/scontrol',     28
dtype: int64
COMMAND
['/usr/bin/scontrol',    336
dtype: int64
COMMAND
['/usr/bin/scontrol',    288
['/usr/bin/sbatch',       24
dtype: int64
COMMAND
['/usr/bin/sbatch',    302
dtype: int64
COMMAND
['/usr/bin/scontrol',    287
['/usr/bin/sbatch',        3
dtype: int64
COMMAND
['/usr/bin/scontrol',    205
['/usr/bin/sbatch',       63
dtype: int64
COMMAND
['/usr/bin/scontrol',    262
dtype: int64
COMMAND
['/usr/bin/scontrol',    253
dtype: int64
COMMAND
['/usr/bin/scontrol',    250
dtype: int64
COMMAND
['/usr/bin/scontrol',    130
['/usr/bin/sbatch',      120
dtype: int64
COMMAND
['/usr/bin/scontrol',    183
['/usr/bin/sbatch',       63
dtype: int64
COMMAND
['/usr/bin/scontrol',    246
dtype: int64
COMMAND
['/usr/bin/scontrol',    202
['/usr/bin/sbatch',       43
dtyp